In [1]:
from datascience import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
%matplotlib inline
plt.style.use('ggplot')

# import clustering methods from `sklearn`
import sklearn.cluster as skcluster

# Initial forays into the canonicalization problem

As with any problem where you start off having no idea what to do, exploratory data analysis is required.

In [ ]:
dat = 